In [1]:
import numpy as np
from bonebox.phantoms.TrabeculaeVoronoi_Shi import *
import ipyvolume as ipv
import nibabel as nib
import random
import math
import os
from tqdm import tqdm

# Parameters for generating phantom mesh
# volume extent in XYZ (mm), number of seeds along XYZ
Sxyz                = (5,5,5)
edgesRetainFraction = 0.5
facesRetainFraction = 0.5
randState           = 123 # for repeatability
# randState           = random.choice(range(100))
volumeSizeVoxels    = (250,250,250)
# Parameters for generating phantom volume
voxelSize           = np.array(Sxyz) / np.array(volumeSizeVoxels)

if not os.path.exists('BoneSet'):
    os.mkdir('BoneSet')

for dilationRadius in [4,5,6,7]:
    for zS in [0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.7,0.9,1.1]:
        for aspect in [1,2,3,4,5,6,7,8]:
            for Rxyz in [0.4]:
                zSt           = zS + dilationRadius * 2 * voxelSize[1]
                Spacingxyz   = (zSt, zSt*aspect, zSt)

                multiplier       = int(np.round(zSt*aspect*5/Sxyz[1],0))
                if multiplier == 0:
                    multiplier = 2
                
                if multiplier == 1:
                    multiplier = 2
                    

                Sxyzt             = (Sxyz[0]*multiplier,Sxyz[1]*multiplier,Sxyz[2]*multiplier)
                volumeSizeVoxelst = (volumeSizeVoxels[0]*multiplier,
                                    volumeSizeVoxels[1]*multiplier,
                                    volumeSizeVoxels[2]*multiplier)

                ################# Generate faces and edges #######################################
                # points = makeSeedPointsCartesian(dx, dy, dz, axy, axz, ayx, ayz, azx, azy, Sxyz)
                points = makeSeedPointsCartesian_LineorArc(Spacingxyz[0], \
                                                           Spacingxyz[1], \
                                                           Spacingxyz[2], \
                                                           0, 0, 0, \
                                                           0, 0, 0, \
                                                           Sxyzt,10000000)

                # Random perturbation
                ppoints = perturbSeedPointsCartesianUniformXYZ(points, Rxyz, randState=randState)

                # Generate voronoi tessilation
                vor, ind = applyVoronoi(ppoints, Sxyzt)

                # Finalize initial edges and faces
                uniqueEdges, uniqueFaces = findUniqueEdgesAndFaces(vor, ind,Sxyzt)
                #################################################################################



                ######################### Filter random edges and faces ########################
                uniqueFacesRetain, facesRetainInd = filterFacesRandomUniform(uniqueFaces, 
                                                                             facesRetainFraction, 
                                                                             randState=randState)
                #################################################################################



                #################### Remove Edge that is part of a face #########################
#                 for j in tqdm(range(len(uniqueFacesRetain))):
#                     temp = [np.sum(np.isin(uniqueEdges[x], uniqueFacesRetain[j]).astype(int)) == 2 for x in range(len(uniqueEdges))]
#                     temp = np.array(temp).astype(int)
#                     if j == 0:
#                         t = temp
#                     else:
#                         t = t + temp
#                 edge = np.delete(uniqueEdges,np.where(t>0)[0].astype(int), axis=0)
                #################################################################################
                edge = uniqueEdges
                #################################################################################




                #################### Randomly Remove Edge #######################################
                uniqueEdgesRetain, edgesRetainInd = filterEdgesRandomUniform(edge, 
                                                                             edgesRetainFraction, 
                                                                             randState=randState)
                #################################################################################




                ####################### PRINT INFO ##############################################
                print("Face number final = %d" % len(facesRetainInd))
                print("Edge number before random removal = %d" % len(edge))
                print("Edge number final = %d\n" % len(edgesRetainInd))
                RodToPlate = len(edge)/len(facesRetainInd)
                print("Max Rod/plate number = %f" % RodToPlate)
                RodToPlate = len(edgesRetainInd)/len(facesRetainInd)
                print("Final Rod/plate number = %f" % RodToPlate)
                #################################################################################




                ############## Generate Volume base on the remaining face and edges #############
                volumeEdges = makeSkeletonVolumeEdges(vor.vertices, 
                                                      uniqueEdgesRetain, 
                                                      voxelSize, 
                                                      volumeSizeVoxelst)
                volumeFaces = makeSkeletonVolumeFaces(vor.vertices, 
                                                      uniqueFacesRetain, 
                                                      voxelSize, 
                                                      volumeSizeVoxelst)
                #################################################################################


                center       = volumeSizeVoxelst[1]/2
                startInd     = int(center-volumeSizeVoxelst[0]/multiplier/2)
                endInd       = int(startInd+volumeSizeVoxelst[0]/multiplier)

                volumeEdges = volumeEdges[startInd:endInd,startInd:endInd,startInd:endInd]
                volumeFaces = volumeFaces[startInd:endInd,startInd:endInd,startInd:endInd]

                volumeDilated = dilateVolumeSphereUniform(np.logical_or(volumeEdges,volumeFaces), \
                                                          dilationRadius)




                ############################### SAVE #############################################
                volumeDilated = volumeDilated * 1;
                ni_img = nib.Nifti1Image(volumeDilated, affine=np.eye(4))
                nib.save(ni_img, os.path.join('BoneSet', 'dilation_' + \
                                                                     str("%0.2f" % (float(dilationRadius)*voxelSize[1]*2)+ \
                                                          '_spacing_' + \
                                                                     str("%0.2f" % zSt)+ \
                                                          '_Disturbance_' + \
                                                                     str("%0.4f" % Rxyz)+ \
                                                          '_aspect_' + \
                                                                     str("%0.0f" % aspect)+ '.nii')))
                #################################################################################
                #################################################################################

Face number final = 27104
Edge number before random removal = 99247
Edge number final = 49624

Max Rod/plate number = 3.661710
Final Rod/plate number = 1.830874
Face number final = 13502
Edge number before random removal = 50111
Edge number final = 25056

Max Rod/plate number = 3.711376
Final Rod/plate number = 1.855725
Face number final = 8146
Edge number before random removal = 30659
Edge number final = 15330

Max Rod/plate number = 3.763688
Final Rod/plate number = 1.881905
Face number final = 5628
Edge number before random removal = 21329
Edge number final = 10664

Max Rod/plate number = 3.789801
Final Rod/plate number = 1.894812
Face number final = 4804
Edge number before random removal = 18541
Edge number final = 9270

Max Rod/plate number = 3.859492
Final Rod/plate number = 1.929642
Face number final = 13271
Edge number before random removal = 49360
Edge number final = 24680

Max Rod/plate number = 3.719388
Final Rod/plate number = 1.859694
Face number final = 11762
Edge number 

Face number final = 8098
Edge number before random removal = 30780
Edge number final = 15390

Max Rod/plate number = 3.800939
Final Rod/plate number = 1.900469
Face number final = 13191
Edge number before random removal = 49510
Edge number final = 24755

Max Rod/plate number = 3.753317
Final Rod/plate number = 1.876658
Face number final = 12020
Edge number before random removal = 45878
Edge number final = 22939

Max Rod/plate number = 3.816805
Final Rod/plate number = 1.908403
Face number final = 18017
Edge number before random removal = 67914
Edge number final = 33957

Max Rod/plate number = 3.769440
Final Rod/plate number = 1.884720
Face number final = 3488
Edge number before random removal = 13092
Edge number final = 6546

Max Rod/plate number = 3.753440
Final Rod/plate number = 1.876720
Face number final = 1372
Edge number before random removal = 5352
Edge number final = 2676

Max Rod/plate number = 3.900875
Final Rod/plate number = 1.950437
Face number final = 3552
Edge number bef

MemoryError: Unable to allocate 42.4 GiB for an array with shape (2250, 2250, 2250) and data type int32